In [109]:
# simulating amm in with different fee amounts
from functools import total_ordering
import pandas as pd
import json
import math
from datetime import datetime

In [110]:
def pretty_print_table(df):
    formatted_df = df.copy()  # Create a copy to avoid modifying the original DataFrame
    for col in formatted_df.select_dtypes(include=['float', 'int']).columns:
        formatted_df[col] = formatted_df[col].apply(lambda x: f"{x:,.4f}")
    print(formatted_df)

In [123]:
moe_asset = "USD"

time_interval = "5min"  # Choose from "1min", "5min", "30min", "daily"
asset_list = ["GBP"]

c = 999999999999999
print(c)
total_pool = 1000000

# dont change
specified_weights = {"GBP": 0.5}

999999999999999


In [124]:
time_col = "Local Time"
time_period_price_col = "Open"

# Define date ranges for each time interval
date_ranges = {
    "1min": ("22sep", "22dec"),
    "5min": ("22sep", "22dec"),
    "30min": ("22dec2023", "22dec"),
    "daily": ("22dec2004", "22dec"),
}

# Base directory for the historical data files
base_dir = "hist_bidask"

# Validate time interval and get the date range
if time_interval not in date_ranges:
    raise ValueError(f"Invalid time interval: {time_interval}. Choose from {list(date_ranges.keys())}.")

start_date, end_date = date_ranges[time_interval]

# Create a function to construct file paths dynamically
def get_file_path(asset, interval):
    return f"{base_dir}/{asset}_{start_date}_{end_date}_{interval}.xlsx"

# Load data for each asset
asset_data = {}
for asset in asset_list:
    file_path = get_file_path(asset, time_interval)
    print(f"Loading data from: {file_path}")  # Debug: Show the file path
    # Load the data
    data = pd.read_excel(file_path)

    # Drop unnecessary columns if they exist
    columns_to_drop = ["Local Date", "Refresh Rate", "BidNet"]
    data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], errors="ignore")
    
    # Sort by date in ascending order and reset the index
    data = data.sort_values(by=time_col, ascending=True).reset_index(drop=True)

    # Handle special case for CHF (invert exchange rates)
    #if asset in ["CHF", "CAD", "JPY"]: # 1 currency buys x of usd
    if asset in ["AUD", "NZD", "EUR", "GBP"]: # 1 USD buys x of currency
        for col in ["Bid", "Ask", "High", "Low", "Open"]:
            if col in data.columns:
                data[col] = 1 / data[col]
    
    # Store the processed data
    asset_data[asset] = data
    
    # Debug: Verify the sorted data
    print(f"Asset: {asset}")
    print(asset_data[asset])  # Show the first few rows

# Example: Access EUR data
# eur_data = asset_data["EUR"]
# print(eur_data.head())  # Verify the loaded data

Loading data from: hist_bidask/GBP_22sep_22dec_5min.xlsx
Asset: GBP
               Local Time       Bid       Ask      High       Low      Open
0     2024-09-23 00:35:00  0.750976  0.750525  0.750976  0.750976  0.750976
1     2024-09-23 00:45:00  0.750976  0.750525  0.750976  0.750976  0.750976
2     2024-09-23 00:50:00  0.750976  0.750469  0.750920  0.751033  0.750976
3     2024-09-23 00:55:00  0.751145  0.750638  0.750920  0.751145  0.750920
4     2024-09-23 01:00:00  0.751089  0.750582  0.751089  0.751089  0.751089
...                   ...       ...       ...       ...       ...       ...
18780 2024-12-20 23:40:00  0.795988  0.795545  0.795672  0.796052  0.795798
18781 2024-12-20 23:45:00  0.795862  0.795418  0.795672  0.796052  0.795988
18782 2024-12-20 23:50:00  0.795988  0.795482  0.795545  0.795988  0.795672
18783 2024-12-20 23:55:00  0.795735  0.795608  0.795735  0.796052  0.795735
18784 2024-12-21 00:00:00  0.795862  0.795292  0.795608  0.795925  0.795798

[18785 rows x 6 col

In [125]:
# Merge unique timestamps from both datasets and sort them
#unique_times = pd.concat([aapl_data['Time'], msft_data['Time']]).drop_duplicates().sort_values()
# Extract unique times from all assets dynamically
unique_times = pd.concat([data[time_col] for data in asset_data.values()]).drop_duplicates().sort_values()

# Print the sorted unique times for verification
print("Unique Times:")
print(unique_times)

Unique Times:
0       2024-09-23 00:35:00
1       2024-09-23 00:45:00
2       2024-09-23 00:50:00
3       2024-09-23 00:55:00
4       2024-09-23 01:00:00
                ...        
18780   2024-12-20 23:40:00
18781   2024-12-20 23:45:00
18782   2024-12-20 23:50:00
18783   2024-12-20 23:55:00
18784   2024-12-21 00:00:00
Name: Local Time, Length: 18785, dtype: datetime64[ns]


In [126]:
def find_indexes_by_dates(df, start_date, end_date):
    # Convert the input dates to datetime
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    
    # Find the start index (earliest time not earlier than the start date)
    start_index = df.searchsorted(start_date, side='left')
    # Find the end index (latest time not exceeding the end date)
    end_index = df.searchsorted(end_date, side='right') - 1
    
    # Ensure the indexes are within valid range
    if start_index >= len(df):
        raise ValueError(f"Start date {start_date} is beyond the available data range.")
    if end_index < 0:
        raise ValueError(f"End date {end_date} is earlier than the available data range.")
    
    # Extract neighborhood entries
    start_neighbors = df[max(start_index - 2, 0): start_index + 3]
    end_neighbors = df[max(end_index - 2, 0): end_index + 3]
    
    # Return the indexes and neighborhood entries
    return start_index, end_index, start_neighbors, end_neighbors

# Example usage
start_date = "2024-11-01 00:00:00"
end_date = "2024-12-01 00:00:00"
start_idx, end_idx, start_neighbors, end_neighbors = find_indexes_by_dates(unique_times, start_date, end_date)

# Print results
print(f"Start index: {start_idx}, corresponding time: {unique_times.iloc[start_idx]}")
print(f"Start neighbors:\n{start_neighbors}")
print(f"End index: {end_idx}, corresponding time: {unique_times.iloc[end_idx]}")
print(f"End neighbors:\n{end_neighbors}")


Start index: 8376, corresponding time: 2024-11-01 00:00:00
Start neighbors:
8374   2024-10-31 23:50:00
8375   2024-10-31 23:55:00
8376   2024-11-01 00:00:00
8377   2024-11-01 00:05:00
8378   2024-11-01 00:10:00
Name: Local Time, dtype: datetime64[ns]
End index: 14446, corresponding time: 2024-11-30 00:00:00
End neighbors:
14444   2024-11-29 23:50:00
14445   2024-11-29 23:55:00
14446   2024-11-30 00:00:00
14447   2024-12-01 21:05:00
14448   2024-12-01 21:10:00
Name: Local Time, dtype: datetime64[ns]


In [115]:
# Define manual start and end indices (default 0 for full range)
manual_start_index = 8376  # Set to desired index, e.g., 4
# manual_end_index = len(unique_times) - 10000  # Set to desired index, e.g., 10
manual_end_index = 14446

# Validate and adjust the indices if out of range
if manual_start_index < 0 or manual_start_index >= len(unique_times):
    manual_start_index = 0
if manual_end_index < 0 or manual_end_index >= len(unique_times):
    manual_end_index = len(unique_times) - 1

# Get the corresponding times for the selected indices
period_start = unique_times.iloc[manual_start_index]
period_end = unique_times.iloc[manual_end_index]

# Filter unique times within the selected range
filtered_times = unique_times.iloc[manual_start_index:manual_end_index + 1].reset_index(drop=True)

# Print the selected period and filtered times for verification
print(f"Simulation Period: {period_start} to {period_end}")
print(filtered_times)

# Temporarily set pandas options to display all rows
#with pd.option_context('display.max_rows', None):   print(filtered_times)

Simulation Period: 2024-11-01 00:00:00 to 2024-11-30 00:00:00
0      2024-11-01 00:00:00
1      2024-11-01 00:05:00
2      2024-11-01 00:10:00
3      2024-11-01 00:15:00
4      2024-11-01 00:20:00
               ...        
6113   2024-11-29 23:40:00
6114   2024-11-29 23:45:00
6115   2024-11-29 23:50:00
6116   2024-11-29 23:55:00
6117   2024-11-30 00:00:00
Name: Local Time, Length: 6118, dtype: datetime64[ns]


In [116]:
# data = {"EUR": eur_data} 
# print(data)

print("Used DataFrames for Simulation:")

filtered_asset_data = {}
for asset in asset_list:
    # Get the original data frame for the asset
    df = asset_data[asset]
    
    # Filter rows where the time column matches the filtered times
    filtered_df = df[df[time_col].isin(filtered_times)].reset_index(drop=True)
    filtered_asset_data[asset] = filtered_df
    
    # Print the asset name and the filtered data
    print(f"Asset: {asset}")
    with pd.option_context('display.max_rows', None):   print(filtered_df)
    #print(filtered_df)
    print()  # Add space between assets for clarity

Used DataFrames for Simulation:
Asset: EUR
              Local Time       Bid       Ask      High       Low      Open
0    2024-11-01 00:00:00  0.918949  0.918780  0.918949  0.919033  0.918949
1    2024-11-01 00:05:00  0.918864  0.918780  0.918864  0.919033  0.919033
2    2024-11-01 00:10:00  0.918864  0.918780  0.918780  0.918864  0.918864
3    2024-11-01 00:15:00  0.918864  0.918780  0.918864  0.918864  0.918864
4    2024-11-01 00:20:00  0.918949  0.918864  0.918864  0.919033  0.918864
5    2024-11-01 00:25:00  0.918949  0.918864  0.918949  0.919033  0.919033
6    2024-11-01 00:30:00  0.918949  0.918780  0.918949  0.919033  0.919033
7    2024-11-01 00:35:00  0.918864  0.918780  0.918864  0.918864  0.918864
8    2024-11-01 00:40:00  0.918780  0.918695  0.918780  0.918864  0.918864
9    2024-11-01 00:45:00  0.918695  0.918611  0.918695  0.918780  0.918780
10   2024-11-01 00:50:00  0.918780  0.918695  0.918695  0.918780  0.918695
11   2024-11-01 00:55:00  0.918695  0.918611  0.918695  0

In [117]:
usd_weight = 1 - sum(specified_weights.values())

# Check if USD weight is valid
if usd_weight <= 0:
    raise ValueError(
        f"Invalid weights: the specified weights ({sum(specified_weights.values())}) "
        f"exceed or equal 1, leaving no positive weight for USD."
    )

# Add USD to the list of assets and its weight to the weights dictionary
assets = [moe_asset] + asset_list
weights = {moe_asset: usd_weight, **specified_weights}

# Verify weights sum to 1 (debugging and validation)
if abs(sum(weights.values()) - 1) > 1e-9:
    raise ValueError(
        f"Weights do not sum to 1. Current total: {sum(weights.values())}."
    )

# Fetch spot prices for each asset
# USD is always 1 (medium of exchange), others fetch dynamically
'''spot_prices = {
    moe_asset: 1,  # USD as the medium of exchange
    "EUR": eur_data.iloc[0][time_period_price_col],
}'''
# Dynamically fetch spot prices for each asset
spot_prices = {}
for asset in assets:
    if asset == moe_asset:
        # Medium of exchange has a fixed spot price of 1
        spot_prices[asset] = 1
    else:
        # Fetch the spot price from the first entry of the corresponding filtered data frame
        spot_prices[asset] = filtered_asset_data[asset].iloc[0][time_period_price_col]


starting_table = pd.DataFrame({
    "asset": assets,
    "weight": [weights[asset] for asset in assets],
    "spot": [spot_prices.get(asset, 1) for asset in assets],
})

starting_table["value"] = total_pool * starting_table["weight"]
starting_table["balance"] = starting_table["value"] * starting_table["spot"]

starting_table.set_index("asset", inplace=True)

pretty_print_table(starting_table)

       weight    spot         value       balance
asset                                            
USD    0.5000  1.0000  500,000.0000  500,000.0000
EUR    0.5000  0.9189  500,000.0000  459,474.3613


In [118]:
# Fee values for different scenarios
#fee_values = [0.1, 0.25, 0.3, 0.4, 0.5, 0.75, 0.9, 1, 1.25, 1.5, 2, 2.5]
fee_values = [0.01, 0.1, 0.5, 1]
order_sizes = [1, 10, 100, 1000]

base_asset = starting_table.index[0]
if True:
  print(f"Base asset: {base_asset}")

  for asset in starting_table.index:
    print(asset)

  # initial price range def
  p = starting_table.at[starting_table.index[1], "spot"]
  sqrt_Pn = math.sqrt(p / c)
  sqrt_Px = math.sqrt(p * c)

  print(f"Initial spot rate p = {p} with the price range multiplier c = {c}")
  print(f"Minimum price = {sqrt_Pn**2} and Maximum price = {sqrt_Px**2}")

Base asset: USD
USD
EUR
Initial spot rate p = 0.9189487226612755 with the price range multiplier c = 2
Minimum price = 0.4594743613306377 and Maximum price = 1.8378974453225507


In [119]:
#Calculate invariant based on balances and weights.
def calculate_invariant(inv_table):
    #inv = 1
    #for asset in assets:
    #    inv *= balances[asset] ** weights[asset]
    #for asset in assets_table.items():
        #inv *= asset["balance"] ** asset["weight"]
    #return inv
    return (inv_table["balance"] ** inv_table["weight"]).prod()

In [120]:
def print_pool_standing(info_table):
    #pd.DataFrame(info_table)
    print(f"Assets: {info_table.index}")
    #print(f"Weights: {weights}")
    print(f"Balances: {info_table["balance"]}")
    print(f"Inv: {calculate_invariant(info_table)}")
    #print("Values: ")
    total_pool_value = 0
    #for asset in assets:
        #total_pool_value += balances[asset] * opens[asset]
        #print("Asset ", asset, " value: ", balances[asset] * opens[asset])
        #inv *= balances[asset] ** weights[asset])
    print(f"Total pool value: {info_table["value"].sum()}")

In [121]:
results = []
result_tables = []
direction = {}
#sim_table = starting_table
for order_size in order_sizes:
    print(f"Order Size simulation: {order_size}")
    # Loop through each fee variation
    for fee_percentage in fee_values:
        print(f"Fee % simulation: {fee_percentage}")
        sim_table = starting_table.copy()
        trades = 0
        fees_earned = 0
        fee = fee_percentage / 100
        # Initialize balances and invariant
        #balances = {asset: (initial_pool * weights[asset]) / spots[asset] for asset in assets}


        trades_log = []
        end_of_period_values = []

        # Loop through each unique time period
        # Fetch data for each asset at the current time and update last close prices if needed
        open_prices = {}
        last_close_prices = {}
        #open_prices[asset] = spots["USD"]
        #last_close_prices[asset] = spots["USD"]
        times = 0
        missing_data_log = {}
        for time in filtered_times:
            '''times += 1
            if times > 2:
                break'''
            #if trades > 10:
                #rint("breaking")
                #break

            if False:
                print(time)
                if False:
                    print(sim_table)
            missing_data_assets = set()
            #
            for asset in sim_table.index:
                if asset == base_asset:
                    #open_prices[asset] = spots["USD"]
                    #last_close_prices[asset] = spots["USD"]
                    pass
                else:
                    direction[asset] = None
                    #asset_data = data[asset]
                    #row = asset_data[asset_data[time_col] == time]
                    asset_df = asset_data[asset]
                    row = asset_df[asset_df[time_col] == time]
                    if row.empty:
                        print(f"No data for asset {asset} at time {time}")
                        missing_data_assets.add(asset)  # Flag the asse
                        # Log the missing data for later analysis
                        if time not in missing_data_log:
                            missing_data_log[time] = []
                        missing_data_log[time].append(asset)
                        continue
                    else:
                        #sim_table.at[asset, "spot"] = row[time_period_price_col].values[0]
                        sim_table.at[asset, "spot"] = row["Open"].values[0]
                        sim_table.at[asset, "value"] = sim_table.at[asset, "balance"] / sim_table.at[asset, "spot"]

                        # Define `market_bid` and `market_ask` once for the current time and asset
                        market_bid = row['Bid'].values[0]
                        market_ask = row['Ask'].values[0]
                        
                        sim_table.at[asset, "market_bid"] = market_bid  # Optional: Store for later use
                        sim_table.at[asset, "market_ask"] = market_ask  # Optional: Store for later use

                        if False:
                            print(f"Asset: {asset}")
                            print(row)
                            
                            print(f"1 {moe_asset} (MOE) = bid: {market_bid}, ask: {market_ask}")
                            #print(f"Market bid: {1 / market_bid}, Market ask: {1 / market_ask} (in terms of MOE)")
                    '''if not row.empty:
                        sim_table.at[asset, "spot"] = row['Open'].values[0]
                        last_close_prices[asset] = row['Close'].values[0]
                    else:
                        open_prices[asset] = sim_table.at[asset, "spot"]'''
                        

            # Trading loop for each asset
            continue_trading = True
            while continue_trading:
                continue_trading = False

                for asset in sim_table.index:
                    if asset == base_asset or asset in missing_data_assets:  # Skip flagged assets for this time
                        continue

                    #print(asset)
                    market_bid = sim_table.at[asset, "market_bid"]
                    market_ask = sim_table.at[asset, "market_ask"]

                    '''
                    # Fetch high and low prices for the asset
                    row = asset_df[asset][asset_df[asset][time_col] == time]
                    if not row.empty:
                        #high_price = row['High'].values[0]
                        #low_price = row['Low'].values[0]
                        market_bid = row['Bid'].values[0]
                        market_ask = row['Ask'].values[0]

                        # Calculate bid and ask prices for the current asset
                        other_balances_product = 1
                        for other_asset in assets:
                            if other_asset != asset and other_asset != "USD":
                                #print("other asset: ", other_asset, " balance: ", balances[other_asset], " weight: ", weights[other_asset])
                                other_balances_product *= balances[other_asset] ** weights[other_asset]

                        inv = calculate_invariant(balances, weights)'''
                    #inv = calculate_invariant(sim_table)
                    balance = sim_table.at[asset, "balance"]
                    #weight = sim_table.at[asset, "weight"]

                    moe_balance = sim_table.at[base_asset, "balance"]
                    #moe_weight = sim_table.at[base_asset, "weight"]
                    
                    if False:
                        print(f"Balance: {balance}, weight: {weight}")
                        other_balances_product = inv / (balance ** weight)
                        print(f"Inv: {inv}, Other bal product: {other_balances_product}")
                    #print("other asset balances produt: ", other_balances_product)
                    #print("balance of the asset: ", balances[asset])
                    #print("invariant: ", balances[asset]**weights[asset] * other_balances_product)

                    #bid_amount_wo_fee = abs(((inv / ((balance + order_size) ** weight * other_balances_product)) ** (1 / sim_table.at["USD", "weight"]) - sim_table.at["USD", "balance"]))
                    #ask_amount_wo_fee = abs(((inv / ((balance - order_size) ** weight * other_balances_product)) ** (1 / sim_table.at["USD", "weight"]) - sim_table.at["USD", "balance"]))

                    #bid_amount_wo_fee = sim_table.at["USD", "balance"] - (((sim_table.at["USD", "balance"]**sim_table.at["USD", "weight"] * balance**weight) / ((balance + order_size)**weight)))**(1 / sim_table.at["USD", "weight"])
                    #ask_amount_wo_fee = -sim_table.at["USD", "balance"] + (((sim_table.at["USD", "balance"]**sim_table.at["USD", "weight"] * balance**weight) / ((balance - order_size)**weight)))**(1 / sim_table.at["USD", "weight"])
                    

                    #bid_amount_wo_fee = moe_balance - (((moe_balance**moe_weight * balance**weight) / ((balance + order_size)**weight)))**(1 / moe_weight)
                    #ask_amount_wo_fee = -moe_balance+ (((moe_balance**moe_weight * balance**weight) / ((balance - order_size)**weight)))**(1 / moe_weight)

                    # mamm
                    #bid_amount_wo_fee = balance - (((moe_balance**moe_weight * balance**weight) / ((moe_balance + order_size)**moe_weight)))**(1 / weight)
                    #ask_amount_wo_fee = -balance+ (((moe_balance**moe_weight * balance**weight) / ((moe_balance - order_size)**moe_weight)))**(1 / weight)
                    L = (moe_balance * math.sqrt(balance / moe_balance)) / (1 - 1 / math.sqrt(c))
                    bid_amount_wo_fee = (balance + L * sqrt_Pn) * (1 - (moe_balance + L / sqrt_Px) / (moe_balance + order_size + L / sqrt_Px))
                    ask_amount_wo_fee = (balance + L * sqrt_Pn) * ((moe_balance + L / sqrt_Px) / (moe_balance - order_size + L / sqrt_Px) - 1)
                    
                    if False:
                        print(f"1 {moe_asset} (MOE) = bid: {market_bid}, ask: {market_ask}")
                        print(asset, " bid (wo fee): ", bid_amount_wo_fee)
                        print(asset, " ask (wo fee): ", ask_amount_wo_fee)
                    
                    bid_amount = bid_amount_wo_fee * (1 - fee)
                    ask_amount = ask_amount_wo_fee * (1 + fee)

                    bid_price = bid_amount / order_size
                    ask_price = ask_amount / order_size

                    if False:
                        print(asset, " bid: ", bid_price)
                        print(asset, " ask: ", ask_price)

                    # Sell asset
                    if market_bid > ask_price and direction[asset] != "sell":
                        if balance >= order_size:
                            direction[asset] = "buy"
                            trades += 1
                            if False:
                                print("Trade: ", trades)
                                pretty_print_table(sim_table)
                                #trades_log.append({'Time': time, 'Asset': asset, 'Action': 'Sell', 'Shares': 1, 'Price': ask_price})
                                #print({'Time': time, 'Asset': asset, 'Action': 'Sell', 'Shares': 1, 'Price': ask_price})
                                #trade_printout = {'Time': time, 'Asset': asset, 'Action': 'Sell', 'Quantity': order_size, 'Amount': round(float(ask_amount), 4), 'Price': round(float(ask_amount_wo_fee), 4)}
                                #trade_printout = {'Time': time, "Sold": {ask_amount}, "of": {asset}, "for": {order_size}, "of": {moe_asset}, f"(exch: {ask_price})": None}
                                '''trade_printout = {
                                    "Time": str(time),
                                    "Bought": f"{float(ask_amount):.2f}",
                                    "of": asset,
                                    "for": f"{order_size}",
                                    "of": moe_asset,
                                    "Exchange Rate": f"{float(ask_price):.4f}"
                                }'''
                                trade_printout = f"[At: {time}] Bought {float(ask_amount):.4f} {asset} for {order_size} {moe_asset} (exch: {float(ask_price):.4f})"
                                trades_log.append(trade_printout)
                                print(trade_printout)

                            sim_table.at[base_asset, "balance"] -= order_size
                            sim_table.at[asset, "balance"] += ask_amount

                            sim_table.at[base_asset, "value"] = sim_table.at[base_asset, "balance"]
                            sim_table.at[asset, "value"] = sim_table.at[asset, "balance"] / sim_table.at[asset, "spot"]
                            fees_earned += (ask_amount - ask_amount_wo_fee) / sim_table.at[asset, "spot"]

                            #inv = calculate_invariant(sim_table)
                            if False:
                                print(f"Fee earned: {round((ask_amount - ask_amount_wo_fee) / sim_table.at[asset, "spot"], 4)} {moe_asset} (total: {round(fees_earned, 4)} {moe_asset}), {asset} fee: {ask_amount - ask_amount_wo_fee}, With fee: {round(ask_amount, 4)}, without fee {round(ask_amount_wo_fee, 4)}")
                                pretty_print_table(sim_table)
                                print()
                            continue_trading = True  # Check for more trades within the period
                            break

                    # Buy asset
                    elif market_ask < bid_price and direction[asset] != "buy":
                        if moe_balance >= bid_price:
                            direction[asset] = "sell"

                            if False:
                                trades += 1
                                print("Trade: ", trades)
                                pretty_print_table(sim_table)
                                #trade_printout = {'Time': time, 'Asset': asset, 'Action': 'Buy', 'Quantity': order_size, 'Amount': round(float(bid_amount), 4), 'Price': round(float(bid_price), 4)}
                                #trade_printout = {'Time': time, "Bought": {ask_amount}, "of": {asset}, "for": {order_size}, "of": {moe_asset}, f"(exch: {bid_price})": None}
                                '''trade_printout = {
                                    "Time": str(time),
                                    "Bought": f"{float(bid_amount):.2f}",
                                    "of": asset,
                                    "for": f"{order_size}",
                                    "of": moe_asset,
                                    "Exchange Rate": f"{float(bid_price):.4f}"
                                }'''
                                trade_printout = f"[At: {time}] Sold {float(bid_amount):.4f} {asset} for {order_size} {moe_asset} (exch: {float(bid_price):.4f})"
                                trades_log.append(trade_printout)
                                print(trade_printout)
                            sim_table.at[base_asset, "balance"] += order_size
                            sim_table.at[asset, "balance"] -= bid_amount

                            sim_table.at[base_asset, "value"] = sim_table.at[base_asset, "balance"]
                            sim_table.at[asset, "value"] = sim_table.at[asset, "balance"] / sim_table.at[asset, "spot"]

                            fees_earned += (bid_amount_wo_fee - bid_amount) / sim_table.at[asset, "spot"]

                            #inv = calculate_invariant(balances, weights)
                            if False:
                                #print(f"Fee earned: {round(bid_amount_wo_fee - bid_amount, 4)} (total: {round(fees_earned, 4)}), With fee: {round(bid_amount, 4)}, without fee {round(bid_amount_wo_fee, 4)}")
                                print(f"Fee earned: {round((bid_amount_wo_fee - bid_amount) / sim_table.at[asset, "spot"], 4)} {moe_asset} (total: {round(fees_earned, 4)} {moe_asset}), {asset} fee: {bid_amount_wo_fee - bid_amount}, With fee: {round(bid_amount, 4)}, without fee {round(bid_amount_wo_fee, 4)}")
                                pretty_print_table(sim_table)
                                print()
                            continue_trading = True  # Check for more trades within the period
                            break

            '''for asset in sim_table.index:
                if asset == base_asset:
                    pass
                else:
                    row = asset_df[asset_df[time_col] == time]
                    # sim_table.at[asset, "spot"] = row['Close'].values[0]

                    sim_table.at[asset, "value"] = sim_table.at[asset, "balance"] / sim_table.at[asset, "spot"]'''

            # Calculate end-of-period pool value
            '''pool_value = balances["USD"]
            il_formula_end_value = 1
            for asset in assets:
                if asset != "USD":
                    pool_value += balances[asset] * last_close_prices[asset]
                    il_formula_end_value *= (last_close_prices[asset] / )
            end_of_period_values.append(pool_value)'''
            end_of_period_values.append(sim_table["value"].sum())

        # Record the final pool value for the current fee
        final_pool_value = end_of_period_values[-1]
        results.append({'Order Size': order_size, 'Fee %': fee_percentage, 'Final_Pool_Value': final_pool_value, "Fees": fees_earned, "Trades": trades})
        result_tables.append(sim_table)

# Convert results to DataFrame for comparison
results_df = pd.DataFrame(results)


Order Size simulation: 1
Fee % simulation: 0.01
Fee % simulation: 0.1
Fee % simulation: 0.5
Fee % simulation: 1
Order Size simulation: 10
Fee % simulation: 0.01
Fee % simulation: 0.1
Fee % simulation: 0.5
Fee % simulation: 1
Order Size simulation: 100
Fee % simulation: 0.01
Fee % simulation: 0.1
Fee % simulation: 0.5
Fee % simulation: 1
Order Size simulation: 1000
Fee % simulation: 0.01
Fee % simulation: 0.1
Fee % simulation: 0.5
Fee % simulation: 1


In [122]:
print(f"Base asset: {base_asset}")

print("Starting table (V0):")
pretty_print_table(starting_table)
print("   Value:               ", f"{starting_table['value'].sum():,.4f}")
print()

#print(sim_table)

print("Hold table (VH):")
hold_table = starting_table.copy()
hold_table['spot'] = sim_table['spot']
hold_table['value'] = hold_table['balance'] / hold_table['spot']
pretty_print_table(hold_table)
print("   Value:               ", f"{hold_table['value'].sum():,.4f}")
print()

print("Summary: ")
#print(results_df)
pretty_print_table(results_df)

if False:
  print()
  print()
  print("End tables:")
  table_index = 1
  for result, table in zip(results, result_tables):
      # Print the summary for the table
      print(f"    Table: {table_index}")
      table_index += 1
      print(json.dumps(result, indent=4))
      # Print the table
      pretty_print_table(table)
      print("   Value:               ", f"{table['value'].sum():,.4f}")

Base asset: USD
Starting table (V0):
       weight    spot         value       balance
asset                                            
USD    0.5000  1.0000  500,000.0000  500,000.0000
EUR    0.5000  0.9189  500,000.0000  459,474.3613
   Value:                1,000,000.0000

Hold table (VH):
       weight    spot         value       balance
asset                                            
USD    0.5000  1.0000  500,000.0000  500,000.0000
EUR    0.5000  0.9454  485,986.0320  459,474.3613
   Value:                985,986.0320

Summary: 
    Order Size   Fee % Final_Pool_Value      Fees        Trades
0       1.0000  0.0100     985,745.4615   99.5292  509,721.0000
1       1.0000  0.1000     985,883.0032  237.2401  130,707.0000
2       1.0000  0.5000     985,930.3222  279.7606   41,347.0000
3       1.0000  1.0000     986,059.6538  383.4215   34,712.0000
4      10.0000  0.0100     985,745.4614   99.5346   50,975.0000
5      10.0000  0.1000     985,883.0080  237.2461   13,071.0000
6      1